**The first thing to do in this case is to get the necessary libraries and modules to use it in python.**

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
from urllib.parse import urljoin
import pandas as pd

**After a long research i could find a solution to apply the scrapper. In this case i had to bring up the pdf files of the Audit Reports Resolution of every year. Since the information is stored by year and by Unit Auditor: "Fondos propios" and "Auditoría Interna".**

In [22]:
"""" EXTRACT PDF FOR AUDITORIAS 2018
IN THIS CASE I STARTED WITH 2018 YEAR AND 'FONDOS PROPIOS' SECTION """


for page in range(1,11): # WITH IT INDICATE THE NUMBER OF PAGES TO LOOK FOR THE FILES
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start='
    
    folder_location = r'/Users/leona/Documents/Python/Contraloria/2018' # It creates a folder to save files
    #If there is no such folder, the script will create one automatically
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url+str((page - 1) * 15))
    """"response = requests.get(url) - with this, it indicates the number of pages, 
    but to make it automatic, it adds "*15" because every next page the link adds 15"""
    soup= bs(response.text, "html.parser")     
    links = soup.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
         """"The above part is key because give the 
        parameters to get the information we need"""
        #Name the pdf files using the last portion of each link which are unique in this case
        filename = os.path.join(folder_location,link.a['href'].split('/')[-1]+str(".pdf"))
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link.a['href'])).content)

In [13]:
# This is a try to know what information it can be extracted and what we need to get a dataset.

archivos2018_fondospropios = []

for page in range(1,5):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start=' 
    response = requests.get(url+str((page - 1) * 15))
    
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        institucion = link.find('span').text.split('-')[-1]
        file_name = link.find('span').text
        url = link.a['href']   
       
        
        print((f'''
        Nombre institución: {institucion}
        Nombre resolucion: {file_name}
        link de descarga: {url}
        '''))


        Nombre institución: MINSA 
        Nombre resolucion: RIA-CGR-063-18-AEES-MINSA 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/7558-ria-cgr-063-18-aees-minsa
        

        Nombre institución: ANA 
        Nombre resolucion: RIA-CGR-064-18-AFYC-ANA 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/7559-ria-cgr-064-18-afyc-ana
        

        Nombre institución: VillaSandino 
        Nombre resolucion: RIA-CGR-098-18-AEES-AM-VillaSandino 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8927-ria-cgr-098-18-aees-am-villasandino
        

        Nombre institución: Nandasmo 
        Nombre resolucion: RIA-CGR-099-18-AEES-AM-Nandasmo 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8928-ria-cgr-099-18-aees-am-nandasmo
        

        Nombre institución: Camoapa 
        Nombre resolucion: RIA-CGR-100-18-AFYC-AM-Camoapa 
        link de descarga: /index.php/reposi


        Nombre institución: ENACAL 
        Nombre resolucion: RIA-CGR-1307-18-AFYC-ENACAL 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8960-ria-cgr-1307-18-afyc-enacal
        

        Nombre institución: INETER 
        Nombre resolucion: RIA-CGR-1308-18-AFYC-INETER 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8961-ria-cgr-1308-18-afyc-ineter
        

        Nombre institución: Telpaneca 
        Nombre resolucion: RIA-CGR-1309-18-AFYC-Telpaneca 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8962-ria-cgr-1309-18-afyc-telpaneca
        

        Nombre institución: Larreynaga 
        Nombre resolucion: RIA-CGR-1310-18-AFYC-AM-Larreynaga 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8963-ria-cgr-1310-18-afyc-am-larreynaga
        

        Nombre institución: Potosi 
        Nombre resolucion: RIA-CGR-1311-18-AFYC-AM-Potosi 
        link de descarga: /index.

In [20]:
""" THIS IS TO GET A DATASET OF THE INFORMATION USEFUL TO KNOW 
HOW MANY AUDIT REPORTS RESOLUTION THE CGR PUBLISH.
ADAPTING THE CODE TO EXTRACT THE PDF FILES, IT GET THE INFORMAITION """

archivos2018_fondospropios = []

for page in range(1,11):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        try:
            res_fp_2018 = {
            'institucion' : link.find('span').text.split('-')[-1],
            'file_name' : link.find('span').text,
            'url' : link.a['href']
            }
            archivos2018_fondospropios.append(res_fp_2018)
        except:
            pass
    
print(len(archivos2018_fondospropios))

137


In [21]:
# NOW THAT THE INFORMATION WAS EXTRACTED AS A DICTIONARY, IT CAN BE CONVERTED TO DATAFRAME USING PANDAS

df = pd.DataFrame(archivos2018_fondospropios)
df

,institucion,file_name,url
0,MINSA,RIA-CGR-063-18-AEES-MINSA,/index.php/repositorio/send/77-cgr-fondos-prop...
1,ANA,RIA-CGR-064-18-AFYC-ANA,/index.php/repositorio/send/77-cgr-fondos-prop...
2,VillaSandino,RIA-CGR-098-18-AEES-AM-VillaSandino,/index.php/repositorio/send/77-cgr-fondos-prop...
3,Nandasmo,RIA-CGR-099-18-AEES-AM-Nandasmo,/index.php/repositorio/send/77-cgr-fondos-prop...
4,Camoapa,RIA-CGR-100-18-AFYC-AM-Camoapa,/index.php/repositorio/send/77-cgr-fondos-prop...
...,...,...,...
132,SantoTomas,RIA-CGR-970-18-AFYC-AM-SantoTomas,/index.php/repositorio/send/77-cgr-fondos-prop...
133,SantoDomingo,RIA-CGR-971-18-AFYC-AM-SantoDomingo,/index.php/repositorio/send/77-cgr-fondos-prop...
134,Cardenas,RIA-CGR-972-18-AFYC-AM-Cardenas,/index.php/repositorio/send/77-cgr-fondos-prop...
135,RioBlanco,RIA-CGR-973-18-AFYC-AM-RioBlanco,/index.php/repositorio/send/77-cgr-fondos-prop...


In [ ]:
# SAVE THE DATAFRAME

df.to_csv('CGR_fondospropios2018.csv')

**The next step is repeating the same codes for every year and audit unit as appear in the website section, using each of the link for these sections**